In [3]:
import numpy as np

import openml
import pandas as pd

In [21]:
from tqdm import tqdm

from datasets import load_openml_list, test_dids_classification, valid_large_classification, open_cc_dids, open_cc_valid_dids


In [5]:
%load_ext autoreload

%autoreload 2

### Prepare test datasets

In [6]:
renamer = {'name': 'Name', 'NumberOfFeatures': '# Features', 'NumberOfSymbolicFeatures': '# Categorical Features', 'NumberOfInstances': '# Instances', 'NumberOfMissingValues': '# NaNs', 'NumberOfClasses': '# Classes', 'MinorityClassSize': 'Minority Class Size'}


In [11]:
openml.study.list_suites()

/tmp/ipykernel_153825/1129402949.py:1: FutureWarning: Support for `output_format` of 'dict' will be removed in 0.15 and pandas dataframes will be returned instead. To ensure your code will continue to work, use `output_format`='dataframe'.
  openml.study.list_suites()


{99: {'id': 99,
  'alias': 'OpenML-CC18',
  'main_entity_type': 'task',
  'name': 'OpenML-CC18 Curated Classification benchmark',
  'status': 'active',
  'creation_date': '2019-02-21 18:47:13',
  'creator': 1},
 225: {'id': 225,
  'alias': 'OpenML-friendly',
  'main_entity_type': 'task',
  'name': 'OpenML100-friendly',
  'status': 'active',
  'creation_date': '2019-09-16 19:41:46',
  'creator': 1},
 353: {'id': 353,
  'alias': '8f0ea660163b436bbd4abd49665c7b1d',
  'main_entity_type': 'task',
  'name': 'OpenML-CTR23 - A curated tabular regression benchmarking suite',
  'status': 'active',
  'creation_date': '2023-05-31 16:39:49',
  'creator': 30127}}

In [4]:
suite = openml.study.get_suite(suite_id=99)
tasks = openml.tasks.list_tasks(output_format="dataframe")

In [6]:
# Retrieve task IDs from the suite
suite_task_ids = suite.tasks

# Filter the tasks DataFrame to only include tasks from the suite
suite_tasks = tasks[tasks['tid'].isin(suite_task_ids)]

print(suite_tasks.head())

print(suite_tasks.info())

print(suite_tasks.describe())

    tid                                ttid  did           name  \
1     3  TaskType.SUPERVISED_CLASSIFICATION    3       kr-vs-kp   
4     6  TaskType.SUPERVISED_CLASSIFICATION    6         letter   
9    11  TaskType.SUPERVISED_CLASSIFICATION   11  balance-scale   
10   12  TaskType.SUPERVISED_CLASSIFICATION   12  mfeat-factors   
12   14  TaskType.SUPERVISED_CLASSIFICATION   14  mfeat-fourier   

                    task_type  status     estimation_procedure  \
1   Supervised Classification  active  10-fold Crossvalidation   
4   Supervised Classification  active  10-fold Crossvalidation   
9   Supervised Classification  active  10-fold Crossvalidation   
10  Supervised Classification  active  10-fold Crossvalidation   
12  Supervised Classification  active  10-fold Crossvalidation   

   evaluation_measures source_data target_feature  ...  \
1                  NaN           3          class  ...   
4                  NaN           6          class  ...   
9                  NaN    

In [ ]:


# Loop through the suite task IDs and fetch the dataset for each task
for task_id in suite_task_ids:
    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    
    # Now you can work with the dataset object, e.g., get the data as a pandas DataFrame
    data, _, _, _ = dataset.get_data(target=dataset.default_target_attribute)
    print(f"Task ID: {task_id}")
    print(data.head())  # Print the first few rows of the dataset

/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/tasks/functions.py:376: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  FutureWarning,
/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/datasets/functions.py:444: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  FutureWarning,


Task ID: 3
  bkblk bknwy bkon8 bkona bkspr bkxbq bkxcr bkxwp blxwp bxqsq  ... skrxp  \
0     f     f     f     f     f     f     f     f     f     f  ...     f   
1     f     f     f     f     t     f     f     f     f     f  ...     f   
2     f     f     f     f     t     f     t     f     f     f  ...     f   
3     f     f     f     f     f     f     f     f     t     f  ...     f   
4     f     f     f     f     f     f     f     f     f     f  ...     f   

  spcop stlmt thrsk wkcti wkna8 wknck wkovl wkpos wtoeg  
0     f     f     f     f     f     f     t     t     n  
1     f     f     f     f     f     f     t     t     n  
2     f     f     f     f     f     f     t     t     n  
3     f     f     f     f     f     f     t     t     n  
4     f     f     f     f     f     f     t     t     n  

[5 rows x 36 columns]


/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/tasks/functions.py:376: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  FutureWarning,
/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/datasets/functions.py:444: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  FutureWarning,


Task ID: 6
   x-box  y-box  width  high  onpix  x-bar  y-bar  x2bar  y2bar  xybar  x2ybr  \
0      2      4      4     3      2      7      8      2      9     11      7   
1      4      7      5     5      5      5      9      6      4      8      7   
2      7     10      8     7      4      8      8      5     10     11      2   
3      4      9      5     7      4      7      7     13      1      7      6   
4      6      7      8     5      4      7      6      3      7     10      7   

   xy2br  x-ege  xegvy  y-ege  yegvx  
0      7      1      8      5      6  
1      9      2      9      7     10  
2      8      2      5      5     10  
3      8      3      8      0      8  
4      9      3      8      3      7  
Task ID: 11
   left-weight  left-distance  right-weight  right-distance
0          1.0            1.0           1.0             1.0
1          1.0            1.0           1.0             2.0
2          1.0            1.0           1.0             3.0
3          1.0  

/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/tasks/functions.py:376: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  FutureWarning,
/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/datasets/functions.py:444: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  FutureWarning,
/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/tasks/functions.py:376: FutureWarning: Starting

...

/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/tasks/functions.py:376: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  FutureWarning,
/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/datasets/functions.py:444: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  FutureWarning,


Task ID: 3904
     loc  v(g)  ev(g)  iv(g)      n        v     l      d       i         e  \
0    1.1   1.4    1.4    1.4    1.3     1.30  1.30   1.30    1.30      1.30   
1    1.0   1.0    1.0    1.0    1.0     1.00  1.00   1.00    1.00      1.00   
2   72.0   7.0    1.0    6.0  198.0  1134.13  0.05  20.31   55.85  23029.10   
3  190.0   3.0    1.0    3.0  600.0  4348.76  0.06  17.06  254.87  74202.67   
4   37.0   4.0    1.0    4.0  126.0   599.12  0.06  17.19   34.86  10297.30   

   ...        t  lOCode  lOComment  lOBlank  locCodeAndComment  uniq_Op  \
0  ...     1.30     2.0        2.0      2.0                  2      1.2   
1  ...     1.00     1.0        1.0      1.0                  1      1.0   
2  ...  1279.39    51.0       10.0      8.0                  1     17.0   
3  ...  4122.37   129.0       29.0     28.0                  2     17.0   
4  ...   572.07    28.0        1.0      6.0                  0     11.0   

   uniq_Opnd  total_Op  total_Opnd  branchCount  
0        1

/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/tasks/functions.py:376: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  FutureWarning,
/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/datasets/functions.py:444: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  FutureWarning,
/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/tasks/functions.py:376: FutureWarning: Starting

Task ID: 3917
    loc  v(g)  ev(g)  iv(g)      n       v     l      d      i         e  ...  \
0   1.1   1.4    1.4    1.4    1.3    1.30  1.30   1.30   1.30      1.30  ...   
1   1.0   1.0    1.0    1.0    1.0    1.00  1.00   1.00   1.00      1.00  ...   
2  83.0  11.0    1.0   11.0  171.0  927.89  0.04  23.04  40.27  21378.61  ...   
3  46.0   8.0    6.0    8.0  141.0  769.78  0.07  14.86  51.81  11436.73  ...   
4  25.0   3.0    1.0    3.0   58.0  254.75  0.11   9.35  27.25   2381.95  ...   

         t  lOCode  lOComment  lOBlank  locCodeAndComment  uniq_Op  uniq_Opnd  \
0     1.30     2.0          2        2                  2      1.2        1.2   
1     1.00     1.0          1        1                  1      1.0        1.0   
2  1187.70    65.0         10        6                  0     18.0       25.0   
3   635.37    37.0          2        5                  0     16.0       28.0   
4   132.33    21.0          0        2                  0     11.0       10.0   

   total_Op 

/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/tasks/functions.py:376: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  FutureWarning,
/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/datasets/functions.py:444: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  FutureWarning,
/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/tasks/functions.py:376: FutureWarning: Starting

In [8]:
# Using ``@`` in `pd.DataFrame.query <
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html>`_
# accesses variables outside of the current dataframe.
tasks = tasks.query("tid in @suite.tasks")

In [9]:
tids = list(tasks[np.logical_and(np.logical_and((tasks.NumberOfInstances <= 2000), (tasks.NumberOfFeatures <= 100))
                                 , (tasks.NumberOfClasses <= 10))].tid)

In [10]:
len(tids)

30

In [11]:
tids = list(tasks[tasks.NumberOfInstances <= 2000].tid)

In [12]:
open_cc_dids = [openml.tasks.get_task(task_id).get_dataset().id for task_id in tids]

/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/tasks/functions.py:376: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  FutureWarning,
/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/openml/datasets/functions.py:444: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  FutureWarning,


In [23]:
open_ml_datasets, open_ml_datasets_df = load_openml_list(test_dids_classification, multiclass=True, shuffled=True, filter_for_nan=False, max_samples = 100000, num_feats=100, return_capped=True)


Number of datasets: 36
Loading kr-vs-kp 3 ..
Loading mfeat-factors 12 ..
Loading credit-g 31 ..
Loading vehicle 54 ..
Loading wine 973 ..
Loading kc1 1067 ..
Loading KDDCup09_appetency 1111 ..
Loading airlines 1169 ..
Loading bank-marketing 1461 ..
Loading blood-transfusion-service-center 1464 ..
Loading cnae-9 1468 ..
Loading nomao 1486 ..
Loading phoneme 1489 ..
Loading adult 1590 ..
Loading covertype 1596 ..
Loading numerai28.6 23517 ..
Loading connect-4 40668 ..
Loading car 40975 ..
Loading Australian 40981 ..
Loading segment 40984 ..
Loading jungle_chess_2pcs_raw_endgame_complete 41027 ..
Loading APSFailure 41138 ..
Loading christine 41142 ..
Loading jasmine 41143 ..
Loading sylvine 41146 ..
Loading albert 41147 ..
Loading MiniBooNE 41150 ..
Loading guillermo 41159 ..
Loading riccardo 41161 ..
Loading dilbert 41163 ..
Loading fabert 41164 ..
Loading robert 41165 ..
Loading volkert 41166 ..
Loading dionis 41167 ..
Loading jannis 41168 ..
Loading helena 41169 ..


In [24]:
open_ml_datasets_df = open_ml_datasets_df[open_ml_datasets_df.NumberOfInstances > 10000]

In [25]:
print_table = open_ml_datasets_df
print_table = print_table[['name', 'NumberOfFeatures', 'NumberOfSymbolicFeatures', 'NumberOfInstances', 'NumberOfClasses', 'NumberOfMissingValues', 'MinorityClassSize']].copy()
print_table['id'] = print_table.index
print_table[['NumberOfFeatures', 'NumberOfSymbolicFeatures', 'NumberOfInstances', 'NumberOfClasses', 'NumberOfMissingValues', 'MinorityClassSize']] = print_table[['NumberOfFeatures', 'NumberOfSymbolicFeatures', 'NumberOfInstances', 'NumberOfClasses', 'NumberOfMissingValues', 'MinorityClassSize']].astype(int)
print_table = print_table.rename(columns=renamer)
print(print_table.to_latex(index=False))

\begin{tabular}{lrrrrrrr}
\toprule
                                  Name &  \# Features &  \# Categorical Features &  \# Instances &  \# Classes &  \# NaNs &  Minority Class Size &    id \\
\midrule
                    KDDCup09\_appetency &         231 &                      39 &        50000 &          2 & 8024152 &                  890 &  1111 \\
                              airlines &           8 &                       5 &       539383 &          2 &       0 &               240264 &  1169 \\
                        bank-marketing &          17 &                      10 &        45211 &          2 &       0 &                 5289 &  1461 \\
                                 nomao &         119 &                      30 &        34465 &          2 &       0 &                 9844 &  1486 \\
                                 adult &          15 &                       9 &        48842 &          2 &    6465 &                11687 &  1590 \\
                             covertype &    

### Prepare Validation datasets

In [26]:
open_cc_datasets, open_cc_datasets_df = load_openml_list(open_cc_dids, multiclass=True, shuffled=True, filter_for_nan=False, max_samples = 2000, num_feats=100, return_capped=True)

def extend_datasets(datasets, filtering = False):
    extended_datasets = {}
    i = 0
    for d in tqdm(datasets):
        if ((not 'NumberOfFeatures' in datasets[d])
                or (not 'NumberOfClasses' in datasets[d])
                or (not 'NumberOfInstances' in datasets[d])
                # or datasets[d]['NumberOfFeatures'] >= num_feats
                or datasets[d]['NumberOfClasses'] <= 0):
            print(datasets[d])
            continue
        ds = openml.datasets.get_dataset(d, download_data=False)
        if filtering and (datasets[d]['NumberOfInstances'] < 150
                          or datasets[d]['NumberOfInstances'] > 2000
                         or datasets[d]['NumberOfFeatures'] > 100
                         or datasets[d]['NumberOfClasses'] > 10):
            continue
        extended_datasets[d] = datasets[d]
        extended_datasets[d].update(ds.qualities)
    
    return extended_datasets

# All datasets
openml_list = openml.datasets.list_datasets()
openml_list = pd.DataFrame.from_dict(openml_list, orient="index")

# Select only classification
openml_list = openml_list[~openml_list['MajorityClassSize'].isna()]

# Remove duplicated datasets
duplicated = openml_list.duplicated(subset=['MajorityClassSize', 'MaxNominalAttDistinctValues', 'MinorityClassSize',
       'NumberOfClasses', 'NumberOfFeatures', 'NumberOfInstances',
       'NumberOfInstancesWithMissingValues', 'NumberOfMissingValues',
       'NumberOfNumericFeatures', 'NumberOfSymbolicFeatures'], keep='first')
openml_list = openml_list[~duplicated]

duplicated = openml_list.duplicated(subset=['name'], keep='first')
openml_list = openml_list[~duplicated]

# Filter out datasets that don't have meta information or Don't fulfill other criteria
openml_list = openml_list.to_dict(orient='index')
openml_list = pd.DataFrame.from_dict(extend_datasets(openml_list, filtering=True), orient="index")

# Filter out datasets in Open CC
openml_list = openml_list[~openml_list.name.apply(lambda x: x in test_datasets_multiclass_df.name.values)]
openml_list['CFI'] = openml_list.apply(lambda x: str(x.NumberOfClasses) + '_' + str(x.NumberOfFeatures) + '_' + str(x.NumberOfInstances), axis = 1)
test_datasets_multiclass_df['CFI'] = test_datasets_multiclass_df.apply(lambda x: str(x.NumberOfClasses) + '_' + str(x.NumberOfFeatures) + '_' + str(x.NumberOfInstances), axis = 1)
openml_list = openml_list[~openml_list.CFI.apply(lambda x: x in test_datasets_multiclass_df.CFI.values)]

# Remove time series and artificial data
openml_list = openml_list[~openml_list.name.apply(lambda x: 'autoUniv' in x)]
openml_list = openml_list[~openml_list.name.apply(lambda x: 'fri_' in x)]
openml_list = openml_list[~openml_list.name.apply(lambda x: 'FOREX' in x)]

# Remove datasets that overlapped with Open CC closely by name
openml_list = openml_list[~openml_list.name.apply(lambda x: 'ilpd' in x)]
openml_list = openml_list[~openml_list.name.apply(lambda x: 'car' in x)]
openml_list = openml_list[~openml_list.name.apply(lambda x: 'pc1' in x)]

# Remove datasets that didn't load
openml_list = openml_list[~openml_list.did.apply(lambda x: x in {1065, 40589, 41496, 770, 43097, 43148, 43255, 43595, 43786, 41701})]

# Remove class skew
openml_list = openml_list[(openml_list.MinorityClassSize / openml_list.MajorityClassSize) > 0.05]
openml_list = openml_list[openml_list.AutoCorrelation != 1]

# Remove too easy
openml_list = openml_list[openml_list.CfsSubsetEval_DecisionStumpAUC != 1]

Number of datasets: 30
Loading balance-scale 11 ..
Loading mfeat-fourier 14 ..
Loading breast-w 15 ..
Loading mfeat-karhunen 16 ..
Loading mfeat-morphological 18 ..
Loading mfeat-zernike 22 ..
Loading cmc 23 ..
Loading credit-approval 29 ..
Loading credit-g 31 ..
Loading diabetes 37 ..
Loading tic-tac-toe 50 ..
Loading vehicle 54 ..
Loading eucalyptus 188 ..
Loading analcatdata_authorship 458 ..
Loading analcatdata_dmft 469 ..
Loading pc4 1049 ..
Loading pc3 1050 ..
Loading kc2 1063 ..
Loading pc1 1068 ..
Loading banknote-authentication 1462 ..
Loading blood-transfusion-service-center 1464 ..
Loading ilpd 1480 ..
Loading qsar-biodeg 1494 ..
Loading wdbc 1510 ..
Loading cylinder-bands 6332 ..
Loading dresses-sales 23381 ..
Loading MiceProtein 40966 ..
Loading car 40975 ..
Loading steel-plates-fault 40982 ..
Loading climate-model-simulation-crashes 40994 ..


/home/anna_linne/.pyenv/versions/3.7.12/envs/tabpfn_37/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Support for `output_format` of 'dict' will be removed in 0.15 and pandas dataframes will be returned instead. To ensure your code will continue to work, use `output_format`='dataframe'.
100%|██████████| 1459/1459 [00:09<00:00, 147.57it/s]


NameError: name 'test_datasets_multiclass_df' is not defined

In [ ]:
print_table = openml_list
print_table = print_table[['name', 'NumberOfFeatures', 'NumberOfSymbolicFeatures', 'NumberOfInstances', 'NumberOfClasses', 'NumberOfMissingValues', 'MinorityClassSize']].copy()
print_table['id'] = print_table.index
print_table[['NumberOfFeatures', 'NumberOfSymbolicFeatures', 'NumberOfInstances', 'NumberOfClasses', 'NumberOfMissingValues', 'MinorityClassSize']] = print_table[['NumberOfFeatures', 'NumberOfSymbolicFeatures', 'NumberOfInstances', 'NumberOfClasses', 'NumberOfMissingValues', 'MinorityClassSize']].astype(int)
print_table = print_table.rename(columns=renamer)
print(print_table.to_latex(index=False))